# Create clusters using 60 PCA components (95% variance explained)
1. Create clusters 
    - KMeans
    - Allgo
2. Index
3. Create

In [1]:
# To show plot in line and stop auto scroll
%matplotlib inline

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [1]:
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from scipy.stats import gaussian_kde
from scipy import stats
from scipy.spatial.distance import cdist, pdist, euclidean
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import collections

In [4]:
# Get train and test data 
train = pd.read_csv('./data/train_120feats_Dense_OutlierFree_LogTransform.csv')
test = pd.read_csv('./data/test_119feats_Dense_OutlierFree_LogTransform.csv')

In [8]:
# This function returns the dataframe in clusters.
def putClusterIndex(df, nCluster = 2, method = "KMeans", splitFile = False, writeToFile = False, outPut = 'Train'):
    
    # PCA stage
    print("Perform PCA..")
        # Copy salePrice and save it somewhere else
    if outPut == "Train":
        salePriceCol = df['SalePrice'].copy()
        df.drop(['SalePrice'], axis = 1, inplace = True)
        #  Scale the rest cols
    elif outPut == "Test":
        pass
    else:
        print("You have to choose either 'Test' or 'Train' as output.")
    trainMatrix = df.as_matrix()
    trainMatrix = preprocessing.scale(trainMatrix)
    dfPCA = PCA(n_components=60).fit_transform(trainMatrix) 
    
    # Clustering stage
    print("Use {0} to perform clustering..".format(method))
        # Perform clustering 
    if method == 'KMeans':
        clusterResult = KMeans(n_clusters=nCluster, init='k-means++', n_init=50, max_iter=2000).fit(dfPCA)
    if method == 'Agglo':
        clusterResult = AgglomerativeClustering(n_clusters=nCluster).fit(dfPCA) 
        # Add cluster indexes to the dataframe
    df['Clusterlabel'] = clusterResult.labels_
        # Print out the clusters and items count in each cluster
    clusterStats = collections.Counter(clusterResult.labels_)
    
    # Print out results
    print("Clustering Results:")
    for key, value in clusterStats.items():
        print("There is {0} items in Cluster #{1}".format(key, value))
    
    # Data output
        # Add SalePrice back to the dataframe
        if outPut == "Train":
            df['SalePrice'] = salePriceCol
        # When NOT split files according on cluster index:
    if splitFile == False and writeToFile == True:
        print("Save ONE file to local...")
        print("Save to ./data/FinalRR{0}_{1}_Clustered{2}_Dense_OF_Log.csv".format(method, outPut, nCluster))
        df.to_csv("./data/FinalRR{0}_{1}_Clustered{2}_Dense_OF_Log.csv".format(method, outPut, nCluster),index=False)
        # When DO SPLIT files according to cluster index:
    if splitFile == True and writeToFile == True:
        print("Save MULTIPLE files to local...")
        for cluster in np.unique(clusterResult.labels_):
            temp = df[df['Clusterlabel']==cluster].copy()
            temp.drop(['Clusterlabel'], axis = 1, inplace = True)
            print("Save to ./data/FinalRR{0}_Cluster{1}_{2}_Dense_OF_Log.csv".format(method, cluster, outPut))
            temp.to_csv("./data/FinalRR{0}_Cluster{1}_{2}_Dense_OF_.Log.csv".format(method, cluster, outPut),index=False)
    print("Program Finished!")

In [12]:
putClusterIndex(train,nCluster = 2, method = "KMeans",splitFile = False, writeToFile = True, outPut = 'Train')

Perform PCA..
Use KMeans to perform clustering..
Clustering Results:
There is 0 items in Cluster #608
There is 1 items in Cluster #807
Save ONE file to local...
Save to ./data/FinalRRKMeans_Train_Clustered2_Dense_OF_Log.csv
Program Finished!


In [44]:
train = pd.read_csv('./Data/FinalRRAgglo_Train_Clustered2_Dense_OF_Log.csv')
test = pd.read_csv('./Data/FinalRRAgglo_Test_Clustered2_Dense_OF_Log.csv')
# Convert to sections
X_train, y_train, train_cluster = train.iloc[:, :-2].copy(), train.iloc[:, -1].copy(), train.iloc[:, -2].copy()
X_test, test_cluster = test.iloc[:, :-1].copy(), pd.DataFrame(test.iloc[:, -1].copy())
test_cluster['SalePrice'] = 0

In [46]:
test[test_cluster['Clusterlabel'] == 0]

,MSZoning,LotFrontage,LotArea,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,Clusterlabel
0,2,4.394449,9.360741,2,1,2,1,1,141000.0,2,...,0,0,0,0,0,0,0,0,0,0
1,3,4.406719,9.565775,2,2,2,1,1,141000.0,3,...,0,0,0,0,0,0,0,0,0,0
6,3,4.219508,8.984819,2,2,2,1,1,181000.0,3,...,0,0,0,0,0,0,0,0,0,0
8,3,4.454347,9.227886,2,1,2,1,1,181000.0,3,...,0,0,0,0,0,0,0,0,0,0
9,3,4.262680,9.036106,2,1,2,1,1,141000.0,3,...,0,0,0,0,0,0,0,0,0,0
11,2,3.091042,7.427144,2,1,2,1,1,106000.0,3,...,0,0,0,0,0,0,0,0,0,0
12,2,3.091042,7.427144,2,1,2,1,1,106000.0,3,...,0,0,0,0,0,0,0,0,0,0
13,3,3.218876,7.732369,2,1,2,1,1,146000.0,3,...,0,0,0,0,0,0,0,0,0,0
24,3,4.219508,9.495594,2,2,2,1,1,181000.0,3,...,0,0,0,0,0,0,1,0,0,0
31,2,4.262680,9.190240,2,1,2,1,1,184900.0,2,...,0,0,0,0,0,0,0,0,0,0


In [70]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, AdaBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn import clone
from sklearn import tree
import xgboost as xgb
import numpy as np
import lightgbm as lgb
import tensorflow as tf
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import KFold 
import warnings
from sklearn.svm import SVR



# ~~~~~~~~~~~~~~~~~~ Summary ~~~~~~~~~~~~~~~~~~~~
# This is class that you can create to instantiate a model, be it lasso or xgb.
#
# ~~~~~~~~~~~~~~~~~~ Functions ~~~~~~~~~~~~~~~~~~
#	- def train_validate(): for training and validating only on training set
# 	- def train_predict(): for training on test set and submitting on test set to Kaggle
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
class Model():

	def __init__(self, alpha = .85, random_state = 1, model = "lasso"):
		self.model_name = model

		# Creating the correct model.
		if model == "gboost":
			self.model = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
		elif model == "gboost_deep":
			self.model = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.02,
                                   max_depth=6, max_features='sqrt',
                                   min_samples_leaf=17, min_samples_split=12, 
                                   loss='huber', random_state =5)
		elif model == "ridge":
			self.model = Ridge(alpha = .0005, random_state = 42)
		elif model == "lasso":
			self.model = Lasso(alpha = .0005, random_state = 42)
		elif model == "elastic":
			self.model = ElasticNet(alpha=.0005, l1_ratio=0.9, random_state=42)
		elif model == "rf":
			self.model = RandomForestRegressor(n_estimators = 95, max_depth = 300, random_state = 42)
		elif model == "krr":
			self.model = KernelRidge(alpha=.6, kernel='polynomial', degree=2, coef0=2.5)
		elif model == "xgb":
			self.model = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

		elif model == "xgb_deep":
			self.model = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=7, 
                             min_child_weight=1.7817, n_estimators=4000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
		elif model == "adaboost":
			self.model = AdaBoostRegressor(tree.DecisionTreeRegressor(),
                          n_estimators=500, random_state=42)
		elif model == "svr":
			self.model = SVR(C=10, epsilon=0.0, kernel = 'rbf')
		elif model == "lgb":
			self.model = lgb.LGBMRegressor(
				objective='regression',
				num_leaves=5,
                learning_rate=0.05,
                n_estimators=720,
                max_bin = 55,
                bagging_fraction = 0.8,
                bagging_freq = 5,
                feature_fraction = 0.2319,
                feature_fraction_seed=9,
                bagging_seed=9,
                min_data_in_leaf=6,
                min_sum_hessian_in_leaf=11)
		else:
			raise ValueError('Please give a model that exists in models.py!')

	def train_validate(self, X, y, n_folds = 5):
		# ~~~~~~~~~~~~~~~~~~ Summary ~~~~~~~~~~~~~~~~~~~~
		# Fits lasso to n_folds, computes error across all folds averaged and saves model.
		# Note that this fits on 80% of data in 5 folds
		#
		# ~~~~~~~~~~~~~~~~ Parameters ~~~~~~~~~~~~~~~~~~~
		# Inputs: 
		#	- X: this is the full training set, ~1450 observations
		#	- y: this the labels from the training set, ~1450 labels
		#	- n_folds: number of folds to run for validation
		# Outputs:
		#	- y_pred_all: this is the predictions from each fold combined with each other
		# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

		# Either stratified or not
		skf = KFold(n_splits=n_folds)
		#skf = StratifiedKFold(n_splits=n_folds)

		y_pred_all = np.zeros(len(y))
		log_error_sum = 0

		for train_index, test_index in skf.split(X, y):
			X_train, X_test = X[train_index], X[test_index]
			y_train, y_test = y[train_index], y[test_index]

			# Cloning to avoid residual fits
			instance = clone(self.model)

			# NOTE: If you don't want to transform, comment this.
			# Forward transform (normalizing for effect of national average)
			# y_transform = transform_y(X_train, y_train, direction = "forward")

			# Turn off warnings
			warnings.filterwarnings('ignore')

			# Fitting
			instance.fit(X_train, y_train) #, y_transform)

			# Get predictions
			y_pred = instance.predict(X_test)

			# NOTE: If you don't want to transform, comment this.
			# Reverse to original data (adding effect of year back in)
			# y_pred = transform_y(X_test, y_pred, direction = "reverse")

			# Removing the clone
			del instance

			# Anything less than $0 is put to 50,000
			mask = y_pred < 0
			y_pred[mask] = 50000

			# Adding to error sum.
			log_error_part = get_error(y_test, y_pred)
			log_error_sum += log_error_part

			# Predicting on out of fold
			y_pred_all[test_index] = y_pred
			badBoy = get_error(y_test, y_pred, type = "rmse")
            

		print("Log error across all validation folds for {} is {}".format(self.model_name, get_error(y_test, y_pred, type = "rmse")))
		return y_pred_all, badBoy

	def train_predict(self, X_train, y_train, X_predict):
		# ~~~~~~~~~~~~~~~~~~ Summary ~~~~~~~~~~~~~~~~~~~~
		# This function will train on the entire training dataset and will then create predictions on the Kaggle test set.
		# This function is mean primarily for the purpose of submission.
		#
		# ~~~~~~~~~~~~~~~~ Parameters ~~~~~~~~~~~~~~~~~~~
		# Input:
		#	- X_train: Full train dataset to train on
		#	- y_train: Full y train to train on
		#	- X_predict: Full test dataset (without labels)
		# Output:
		#	- y_pred: predictions on X
		# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
		instance = clone(self.model)
		instance.fit(X_train, y_train)

		y_pred = instance.predict(X_predict)
		return y_pred



# def transform_y(X, y, direction):
# 	# If direction is forward, then we want to scale the costs so that the higher housing market years
# 	# like 2007 have y values that are scaled lower (to normalize for price), while 2009 houses are scaled
# 	# up (to normalize for price).
# 	time_series = pd.read_csv('./Data/time_series.csv')

# 	# Undo log transform
# 	y = np.exp(y)

# 	X_and_y  = np.column_stack((X, y))
# 	X_and_y = pd.DataFrame(X_and_y)

# 	print("merging two dataframes, time series and X_y")
# 	print(X_and_y.shape)
# 	X_and_y = pd.merge(X_and_y, time_series,  how='left', left_on=[61,60], right_on = ['YrSold','MoSold'])
# 	print(X_and_y.shape)



# 	# Important column indices for Roger's data (0 index):
# 	# YrSold: 61
# 	# MoSold: 60
# 	# SalePrice: 119
# 	if direction == "forward":
		
# 		# Redo log transform
# 	if direction == "reverse":

# 		# Redo log transform
# 	# After scaling, remove columns
# 	# index_online
# 	# index_lowest
# 	# index_scaled




# ~~~~~~~~~~~~~~~~~~ Summary ~~~~~~~~~~~~~~~~~~~~
# get_error returns root mean square log error or root mean square error.
# rmsle is good for Kaggle comparison
# rmse is good for seeing the actual average error
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def get_error(y_true, y_pred, type = "rmsle"):
	assert len(y_true) == len(y_pred), "Length of prediction and test do not match!"

	list_types = ["rmsle","rmse"]
	assert type in list_types, "Error type you requested is not an option. Choose 'rmse' or 'rmsle' please."

	# Root mean square error (RMSE)
	if type == "rmse":
		return np.square(y_pred - y_true).mean() ** 0.5

	# Root mean square log error (RMSLE)
	if type == "rmsle":
		return np.square(np.log(y_pred) - np.log(y_true)).mean() ** 0.5



In [82]:
nCluster = 2
import pandas as pd
import numpy as np
def importClusters(nClusters = nCluster, method = "KMeans"):
    # Read raw files
    if method == "KMeans":
        train = pd.read_csv('./Data/FinalRRKMeans_Train_Clustered2_Dense_OF_Log.csv')
        test = pd.read_csv('./Data/FinalRRKMeans_Test_Clustered2_Dense_OF_Log.csv')
    if method == "Agglo":
        train = pd.read_csv('./Data/FinalRRAgglo_Train_Clustered2_Dense_OF_Log.csv')
        test = pd.read_csv('./Data/FinalRRAgglo_Test_Clustered2_Dense_OF_Log.csv')
    # Convert to sections
    X_train, y_train, train_cluster = train.iloc[:, :-2].copy(), train.iloc[:, -1].copy(), pd.DataFrame(train.iloc[:, -2].copy())
    X_test, test_cluster = test.iloc[:, :-1].copy(), pd.DataFrame(test.iloc[:, -1].copy())
    train_cluster['index'] = X_train.index
    test_cluster['index'] = X_test.index
    return X_train, y_train, train_cluster, X_test, test_cluster


X_train_raw, y_train_raw, train_cluster, X_test_raw, test_cluster = importClusters(nClusters = nCluster, method = "KMeans")

In [83]:
def calAverageErr(clusterCount, errorLog):
    total = clusterCount.sum()
    fraction = clusterCount/total
    fracError = errorLog*fraction
    result = fracError.sum()
    return result

In [84]:
print("Cluster with Kmean++")
print("*******************************************")
print("*******************************************")
# Run models
models = ["lasso", "elastic", "lgb", "gboost", "gboost_deep", "xgb", "xgb_deep"]
X_stack_train = np.zeros((y_train.shape[0], len(models)))
X_stack_predict = np.zeros((X_test.shape[0], len(models)))
i = 0
# Train based on clusters

for model in models:
    instance = Model(model = model)
    instance_pred = []
    instance_test_pred = []
    clusterCount = np.zeros(nCluster)
    errorLog = np.zeros(nCluster)
    print("*******************************************")
    print("Train with", model)
    for i in range(nCluster):
        X_train = X_train_raw[train_cluster.Clusterlabel==i].as_matrix()
        y_train = y_train_raw[train_cluster.Clusterlabel==i].as_matrix()
        X_test = X_test_raw[test_cluster.Clusterlabel==i].as_matrix()
        print("Now train cluster {0}, with {1} rows...".format(i, X_train.shape[0]))
        clusterCount[i]=X_train.shape[0]
        result, badBoy = instance.train_validate(X_train, y_train)
        errorLog[i] = badBoy
        instance_pred.append(result)
        
        print("Now predict cluster {0}, with {1} rows...".format(i, X_test.shape[0]))
        instance_test_pred.append(instance.train_predict(X_train, y_train, X_test))
    tError = calAverageErr(clusterCount, errorLog)
    print("Total error with {0} is {1}".format(model,tError))
    print("*******************************************")
        #test_cluster['instance_test_pred'] = instance.train_predict(X_train, y_train, X_predict)
    #X_stack_train[:,i] = test_cluster['instance_pred'].as_matrix()
    #X_stack_predict[:,i] = test_cluster['instance_test_pred'].as_matrix()
    i += 1
print ("finished!")

Cluster with Kmean++
*******************************************
*******************************************
*******************************************
Train with lasso
Now train cluster 0, with 608 rows...
Log error across all validation folds for lasso is 0.11060306978506862
Now predict cluster 0, with 873 rows...
Now train cluster 1, with 807 rows...
Log error across all validation folds for lasso is 0.1247057885133562
Now predict cluster 1, with 586 rows...
Total error with lasso is 0.11864610442374571
*******************************************
*******************************************
Train with elastic
Now train cluster 0, with 608 rows...
Log error across all validation folds for elastic is 0.11068453050604168
Now predict cluster 0, with 873 rows...
Now train cluster 1, with 807 rows...
Log error across all validation folds for elastic is 0.12479951967894488
Now predict cluster 1, with 586 rows...
Total error with elastic is 0.1187345632004112
*******************************

In [85]:
nCluster = 2
import pandas as pd
import numpy as np
def importClusters(nClusters = nCluster, method = "KMeans"):
    # Read raw files
    if method == "KMeans":
        train = pd.read_csv('./Data/FinalRRKMeans_Train_Clustered2_Dense_OF_Log.csv')
        test = pd.read_csv('./Data/FinalRRKMeans_Test_Clustered2_Dense_OF_Log.csv')
    if method == "Agglo":
        train = pd.read_csv('./Data/FinalRRAgglo_Train_Clustered2_Dense_OF_Log.csv')
        test = pd.read_csv('./Data/FinalRRAgglo_Test_Clustered2_Dense_OF_Log.csv')
    # Convert to sections
    X_train, y_train, train_cluster = train.iloc[:, :-2].copy(), train.iloc[:, -1].copy(), pd.DataFrame(train.iloc[:, -2].copy())
    X_test, test_cluster = test.iloc[:, :-1].copy(), pd.DataFrame(test.iloc[:, -1].copy())
    train_cluster['index'] = X_train.index
    test_cluster['index'] = X_test.index
    return X_train, y_train, train_cluster, X_test, test_cluster


X_train_raw, y_train_raw, train_cluster, X_test_raw, test_cluster = importClusters(nClusters = nCluster, method = "Agglo")

In [86]:
print("Cluster with Agglo")
print("*******************************************")
print("*******************************************")
# Run models
models = ["lasso", "elastic", "lgb", "gboost", "gboost_deep", "xgb", "xgb_deep"]
X_stack_train = np.zeros((y_train.shape[0], len(models)))
X_stack_predict = np.zeros((X_test.shape[0], len(models)))
i = 0
# Train based on clusters

for model in models:
    instance = Model(model = model)
    instance_pred = []
    instance_test_pred = []
    clusterCount = np.zeros(nCluster)
    errorLog = np.zeros(nCluster)
    print("*******************************************")
    print("Train with", model)
    for i in range(nCluster):
        X_train = X_train_raw[train_cluster.Clusterlabel==i].as_matrix()
        y_train = y_train_raw[train_cluster.Clusterlabel==i].as_matrix()
        X_test = X_test_raw[test_cluster.Clusterlabel==i].as_matrix()
        print("Now train cluster {0}, with {1} rows...".format(i, X_train.shape[0]))
        clusterCount[i]=X_train.shape[0]
        result, badBoy = instance.train_validate(X_train, y_train)
        errorLog[i] = badBoy
        instance_pred.append(result)
        
        print("Now predict cluster {0}, with {1} rows...".format(i, X_test.shape[0]))
        instance_test_pred.append(instance.train_predict(X_train, y_train, X_test))
    tError = calAverageErr(clusterCount, errorLog)
    print("Total error with {0} is {1}".format(model,tError))
    print("*******************************************")
        #test_cluster['instance_test_pred'] = instance.train_predict(X_train, y_train, X_predict)
    #X_stack_train[:,i] = test_cluster['instance_pred'].as_matrix()
    #X_stack_predict[:,i] = test_cluster['instance_test_pred'].as_matrix()
    i += 1
print ("finished!")

Cluster with Agglo
*******************************************
*******************************************
*******************************************
Train with lasso
Now train cluster 0, with 857 rows...
Log error across all validation folds for lasso is 0.12230668901949009
Now predict cluster 0, with 944 rows...
Now train cluster 1, with 558 rows...
Log error across all validation folds for lasso is 0.11209250754522095
Now predict cluster 1, with 515 rows...
Total error with lasso is 0.1182787644522518
*******************************************
*******************************************
Train with elastic
Now train cluster 0, with 857 rows...
Log error across all validation folds for elastic is 0.12261049796798258
Now predict cluster 0, with 944 rows...
Now train cluster 1, with 558 rows...
Log error across all validation folds for elastic is 0.11220389267674219
Now predict cluster 1, with 515 rows...
Total error with elastic is 0.11850669178246163
********************************